In [2]:
'''
Accidents analytics

by Pinhas Ravinskiy

Python 3.5
'''

# Imports section

import os
import sys
import statistics
from datetime import datetime
from csv import DictReader
from tqdm import tqdm_notebook as tqdm  # if some error/warnings try to run in terminal "jupyter nbextension enable --py --sys-prefix widgetsnbextension" and reload notebook page

print(sys.version)

3.6.2 (v3.6.2:5fd33b5926, Jul 16 2017, 20:11:06) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]


In [3]:
# Data model section

type_custom_datetime = datetime.strptime
DATE_FORMAT = '%d/%m/%Y'
TIME_FORMAT = '%H:%M'


def type_speed_limit(val: str) -> float:
    if val.lower() == 'null' or not val:
        return float('+infinity')
    else:
        return float(val)

        
# Dataset description
columns = {
    'Accident_Index': {'type': str, 'description': 'Unique ID'},
    'Location_Easting_OSGR': {'type': int, 'description': 'Local British coordinates'},
    'Location_Northing_OSGR': {'type': int, 'description': 'Local British coordinates'},
    'Longitude': {'type': float, 'description': 'Longitude'},
    'Latitude': {'type': float, 'description': 'Latitude'},
    'Police_Force': {'type': int, 'description': 'Police force ID'},
    'Accident_Severity': {'type': int, 'description': 'Accident severity: the higher the number, the worse it is'},
    'Number_of_Vehicles': {'type': int, 'description': 'Number of vehicles involved'},
    'Number_of_Casualties': {'type': int, 'description': 'Number of casualties'},
    'Date': {'type': type_custom_datetime, 'description': 'Date in dd/mm/yyyy format', 'format': DATE_FORMAT},
    'Day_of_Week': {'type': int, 'description': 'Day of week: 1 is Monday, 2 is Tuesday, etc'},
    'Time': {'type': type_custom_datetime, 'description': 'Accident GMT/UTC time in HH:MM format','format': TIME_FORMAT},
    'Local_Authority_(District)': {'type': int, 'description': 'Local authority of district'},
    'Local_Authority_(Highway)': {'type': str, 'description': 'Local authority of highway'},
    '1st_Road_Class': {'type': int, 'description': 'First road class: this field is only used for junctions'},
    '1st_Road_Number': {'type': int, 'description': 'First road number: this field is only used for junctions'},
    'Road_Type': {'type': int, 'description': 'Road type: Roundabout, One way, Dual Carriageway, Single carriageway, slip road, unknown'},
    'Speed_limit': {'type': type_speed_limit, 'description': 'Speed limit'},
    'Junction_Detail': {'type': int, 'description': 'Junction detail: Crossroads, roundabouts, private roads, not a junction, etc'},
    'Junction_Control': {'type': int, 'description': 'Junction control: A person, a type of sign, automated, etc'},
    '2nd_Road_Class': {'type': int, 'description': 'Second road class: this field is only used for junctions'},
    '2nd_Road_Number': {'type': int, 'description': 'Second road number: this field is only used for junctions'},
    'Pedestrian_Crossing-Human_Control': {'type': int, 'description': 'Was there a human controller and what type?'},
    'Pedestrian_Crossing-Physical_Facilities': {'type': int, 'description': 'Was it a zebra crossing, or bridge, or another type'},
    'Light_Conditions': {'type': int, 'description': 'Day, night, street lights or not'},
    'Weather_Conditions': {'type': int, 'description': 'Wind, rain, snow, fog'},
    'Road_Surface_Conditions': {'type': int, 'description': 'Wet, snow, ice, flood'},
    'Special_Conditions_at_Site': {'type': int, 'description': 'Was anything broken or defective, e.g. an obscured sign?'},
    'Carriageway_Hazards': {'type': int, 'description': 'Was something in the way, e.g. a pedestrian, another accident, something in the road?'},
    'Urban_or_Rural_Area': {'type': int, 'description': 'Urban or rural area'},
    'Did_Police_Officer_Attend_Scene_of_Accident': {'type': int, 'description': 'Did police officer attend scene of accident'},
    'LSOA_of_Accident_Location': {'type': str, 'description': 'Local service ordering administration of accident location'}
}


In [10]:
# Input section

INPUT_DIR_PATH = '/Users/ravinskiy/Downloads/Accidents'

def get_input_paths(dir_path: str) -> list:
    paths = []
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            filepath = '%s/%s' % (dir_path, filename)
            paths.append(filepath)
    paths.sort(reverse=True)
    return paths


input_paths = get_input_paths(INPUT_DIR_PATH)

records = list()
skip_count = 0
for path in tqdm(input_paths):
    print('Loading data from file: %s'% path)
    csv_reader = DictReader(open(path, 'r'))
    for line in tqdm(csv_reader):
        record = dict()
        for column_name, column_attributes in columns.items():
            cast_format = column_attributes.get('format', '')
            try:
                if not cast_format:
                    record[column_name] = column_attributes['type'](line[column_name])
                else:
                    record[column_name] = column_attributes['type'](line[column_name], cast_format)
            except:
                if column_name == 'Location_Easting_OSGR' and not line[column_name]:
#                     print('Empty location data. Skipping.')
                    record = dict()
                    skip_count += 1
                    break
        if record:
            records.append(record)

print('Loaded records: %s. Skipped records: %s.' % (len(records), skip_count))


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2016.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2015.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2014.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2013.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2012.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2011.csv


Loading data from file: /Users/ravinskiy/Downloads/Accidents/Accidents_2010.csv



Loaded records: 1013084. Skipped records: 34.


In [13]:
# Report section

for column_name, column_attributes in columns.items():
    print()
    print('Column: %s' % column_name)
    column_type = column_attributes['type'].__name__
    print('Column data type: %s' % column_type)
    print('Column data description: %s' % column_attributes['description'])
    if column_type in ['int', 'float', 'type_speed_limit']:
        column = [record[column_name] for record in records]
        median = statistics.median(column)
        mode = statistics.mode(column)
        stdev = statistics.stdev(column)
        min_val = min(column)
        max_val = max(column)
        column_set = set(column)
        print('Min value: %s' % min_val)
        print('Max value: %s' % max_val)
        print('Median: %s' % median)
        print('Mode: %s' % mode)
        print('Standard deviation: %s' % stdev)
        print('Distinct values count: %s' % len(column_set))


Column: Accident_Index
Column data type: str
Column data description: Unique ID

Column: Location_Easting_OSGR
Column data type: int
Column data description: Local British coordinates
Min value: 64950
Max value: 655370
Median: 448406.0
Mode: 533650
Standard deviation: 95301.54691707491
Distinct values count: 249443

Column: Location_Northing_OSGR
Column data type: int
Column data description: Local British coordinates
Min value: 10290
Max value: 1198000
Median: 249382.5
Mode: 181310
Standard deviation: 159745.6977982696
Distinct values count: 297663

Column: Longitude
Column data type: float
Column data description: Longitude
Min value: -7.516225
Max value: 1.759398
Median: -1.2874715
Mode: -0.104426
Standard deviation: 1.4002475650731785
Distinct values count: 826899

Column: Latitude
Column data type: float
Column data description: Latitude
Min value: 49.912941
Max value: 60.662043
Median: 52.1299015
Mode: 51.519764
Standard deviation: 1.438531682596279
Distinct values count: 788300